In [ ]:
#iniciar spark
from pyspark import SparkConf
from pyspark.sql import SparkSession

conf = SparkConf()
#conf.set('spark.jars.packages', 'org.apache.hadoop:hadoop-aws:3.2.2')
conf.set('spark.jars.packages', 'org.apache.hadoop:hadoop-aws:3.2.2,com.microsoft.azure:spark-mssql-connector_2.12:1.2.0')
conf.set('spark.hadoop.fs.s3a.aws.credentials.provider', 'com.amazonaws.auth.InstanceProfileCredentialsProvider')
spark = SparkSession.builder.config(conf=conf).getOrCreate()

In [ ]:
df = spark.read.option('delimiter', ';').option('header', 'true').csv('s3a://marise-3cco/vendas_anuais.csv')

In [ ]:
df.show()

In [ ]:
from pyspark.sql.functions import col
from pyspark.sql.types import IntegerType

_df = df.select(
    col('id_cliente').cast(IntegerType()).alias('ID_CLIENTE'),
    col('total_vendas').cast(IntegerType()).alias('TOTAL_VENDAS'),
    col('numero_pedidos').cast(IntegerType()).alias('NUMERO_PEDIDOS'),
    col('perfil').alias('PERFIL')
)

_df.write \
   .mode('overwrite') \
   .csv('s3a://marise-3cco/vendas_anuais')

_df.write \
 .format('com.microsoft.sqlserver.jdbc.spark') \
 .mode('overwrite') \
 .option('driver', 'com.microsoft.sqlserver.jdbc.SQLServerDriver') \
 .option('url', 'jdbc:sqlserver://bd-dw-01.database.windows.net;databaseName=BD-DW;') \
 .option('dbtable', 'VENDAS_ANUAIS') \
 .option('user', 'urubu100') \
 .option('password', 'Urubu1@@') \
 .save()